In [2]:
# working with, mainly resizing, images
import cv2                 

# dealing with arrays
import numpy as np         

# dealing with directories
import os                  

# mixing up or currently ordered data that might lead our network astray in training.
from random import shuffle 

# a nice pretty percentage bar for tasks. Thanks to viewer Daniel BA1/4hler for this suggestion
from tqdm import tqdm      

# neural networks
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf


TRAIN_DIR = './train'
TEST_DIR = './test'
IMG_SIZE = 64
LR = 1e-3

# just so we remember which saved model is which, sizes must match
MODEL_NAME = 'dogs-vs-cats-{}-{}.model'.format(LR, '2conv-basic') 

def label_img(imgname):
    file_extension = imgname.split('.')[-1]
    if file_extension != 'jpg':
        return []
    word_label = imgname.split('.')[-3]
    # conversion to one-hot array [cat,dog]
    #                            [much cat, no dog]
    if word_label == 'cat': return [1,0]
    #                             [no cat, very doggo]
    elif word_label == 'dog': return [0,1]

    
def create_train_data():
    training_data = []
    img = []
    for imgname in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(imgname)
        if label == []:
            continue
        try:
            path = os.path.join(TRAIN_DIR,imgname)
            img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            training_data.append([np.array(img),np.array(label)])
        except Exception as e:
            print("error with image: '{}', exception: {}".format(imgname, str(e)))
    shuffle(training_data)
    np.save('./train_data_64.npy', training_data)
    return training_data


def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
        
    shuffle(testing_data)
    np.save('./test_data_64.npy', testing_data)
    return testing_data


train_data = create_train_data()
# If you have already created the dataset:
# train_data = np.load('train_data_64.npy')

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 2)

#_________
#convnet = conv_2d(convnet, 32, 5, activation='relu')
#convnet = max_pool_2d(convnet, 5)

#convnet = conv_2d(convnet, 64, 5, activation='relu')
#convnet = max_pool_2d(convnet, 5)

#convnet = conv_2d(convnet, 128, 5, activation='relu')
#convnet = max_pool_2d(convnet, 5)

#convnet = conv_2d(convnet, 64, 5, activation='relu')
#convnet = max_pool_2d(convnet, 5)

#convnet = conv_2d(convnet, 32, 5, activation='relu')
#convnet = max_pool_2d(convnet, 5)
#_________

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')




if os.path.exists('./{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')

    
train = train_data[:-500]
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit(
    {'input': X}, 
    {'targets': Y}, 
    n_epoch=3, 
    validation_set=(
        {'input': test_x}, 
        {'targets': test_y}), 
    snapshot_step=500, 
    show_metric=True, 
    run_id=MODEL_NAME)



Training Step: 564  | total loss: 12.07868 | time: 225.218s
| Adam | epoch: 002 | loss: 12.07868 - acc: 0.4754 -- iter: 11648/24446


KeyboardInterrupt: 

In [ ]:
model.save("./{}".format(MODEL_NAME))